In [178]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

N=50
T=7
R=0.8
M=5
F=0.005

In [179]:
def GetData(file):

    data = pd.read_csv(file)
    data = data[["datadate","tic","adjcp"]]
    return data    

In [180]:
# YYYYMMDD

def PartitionData(Data):
    DateToIndex = {}
    i = 0
    SmallerData = []
    IndexToDay = {}
    
    for date in Data["datadate"].unique():
        day_data = Data[Data["datadate"] == date].copy()
        IndexToDay[i] = date
        SmallerData.append(day_data)
        DateToIndex[date] = i
        i += 1  
    
    return SmallerData, DateToIndex, IndexToDay

In [181]:
temp = PartitionData(GetData("DATA.csv"))

In [182]:
# SmallerData-> horizontal slices in the original data based on date
# DatToIndex-> put in the date to get the index and hence the data for that day

SmallerData = temp[0]
DayToIndex = temp[1]
IndexToDay = temp[2]

In [183]:
# assuming today is one of the days from the data given
def GetMomentumBasedPriority(SmallerData, DayToIndex, today):
    today = int(today)
    k = DayToIndex[today]
    j = k-N

    sum = [0.0]*30
    sum = np.array(sum)
    first=[]

    data = SmallerData[j]
    for value in data["adjcp"]:
        first.append(value)
    
    for it in range(N):
        data = SmallerData[j]
        i=0
        for value in data["adjcp"]:
            sum[i]+=float(value)
            i+=1
        j+=1

    data = SmallerData[k]
    i=0
    ans = np.zeros((30, 2))

    for value in data["adjcp"]:
        div = sum[i]
        temp = (value - first[i]) / div
        ans[i][0] = temp
        ans[i][1] = i
        i+=1
    
    sorted_indices = np.argsort(ans[:, 0])[::-1]
    ans = ans[sorted_indices]
    return ans


In [184]:
def GetBalanced(prices, weights, balance):
    sum=0.0
    for pair in weights:
        id = int(pair[1])
        sum+=prices[id]*pair[0]

    xval = (float(balance))/sum

    ans = np.zeros(30, dtype=float)
    cost=0.0
    for pair in weights:
        id = int(pair[1])
        num = xval*pair[0]
        ans[id] = num
        cost+=ans[id]*prices[id]
            
    return ans

In [185]:
class PortFolio:
    initial=0.0
    cur=0.0
    holdings = np.zeros(30, dtype = float)
    latest_prices = []
    def __init__(self, bal):
        self.initial = bal
        self.current = bal

    # def SellStock(self, index):
    #     self.current+=latest_prices[index]
    #     holdings[index]-=1
        
    def BuyStock(self, index, number):
        self.current-=self.latest_prices[index]*number
        self.holdings[index]+=number

    def CalculateNetWorth(self):
        ans=0.0
        for i in range(30):
            ans+=self.latest_prices[i]*self.holdings[i]
        return ans+self.current
        
    def ChangePricesTo(self,prices):
        self.latest_prices = prices

    def RebalancePortfolio(self, weights):  #weights is array of pairs
        for i in range(30):
            self.current+=self.holdings[i]*self.latest_prices[i] #include fee later
            self.holdings[i] = 0.0
        weight = [w for w in weights if w[0]>0]

        buy = GetBalanced(self.latest_prices, weight, self.current)

        for i in range(30):
            self.BuyStock(i, buy[i])
        

In [192]:
me = PortFolio(10000)

NetWorths = []


for i in range(50,100):
    data = SmallerData[i]
    today = IndexToDay[50]
    prices = []
    for cp in data["adjcp"]:
        prices.append(cp)
    me.ChangePricesTo(prices)
    me.RebalancePortfolio(GetMomentumBasedPriority(SmallerData, DayToIndex, today))
    NetWorths.append(me.CalculateNetWorth())

